In [1]:
import os
import cv2
import dlib
import numpy as np
from sklearn.decomposition import PCA
from sklearn.preprocessing import normalize
from collections import defaultdict
from scipy.spatial.distance import euclidean


In [ ]:
# === Load Face Detector ===
hog_detector = dlib.get_frontal_face_detector()

def augment_image(image):
    augmented = []
    rows, cols = image.shape

    for angle in [-5, 5]:
        M = cv2.getRotationMatrix2D((cols // 2, rows // 2), angle, 1)
        rotated = cv2.warpAffine(image, M, (cols, rows), borderMode=cv2.BORDER_REFLECT)
        augmented.append(rotated)

    for alpha, beta in [(1.1, 5), (0.9, -5)]:
        adjusted = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
        augmented.append(adjusted)

    return augmented

def preprocess_face(img):
    gray = cv2.equalizeHist(img)
    blurred = cv2.GaussianBlur(gray, (3, 3), 0)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    return clahe.apply(blurred)

def load_training_data(png_dataset, target_images_per_person=35):
    label_dict = {}
    current_label = 0
    person_images = {}

    for person in os.listdir(png_dataset):
        person_path = os.path.join(png_dataset, person)
        if not os.path.isdir(person_path):
            continue
        label_dict[current_label] = person
        valid_images = []

        for img_name in os.listdir(person_path):
            img_path = os.path.join(person_path, img_name)
            img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
            if img is None:
                continue

            img = preprocess_face(img)
            faces = hog_detector(img)
            if len(faces) != 1:
                continue

            face = faces[0]
            x, y, w, h = face.left(), face.top(), face.width(), face.height()
            x = max(0, x)
            y = max(0, y)
            x2 = min(img.shape[1], x + w)
            y2 = min(img.shape[0], y + h)
            face_img = img[y:y2, x:x2]

            if face_img.size == 0 or face_img.shape[0] < 50 or face_img.shape[1] < 50:
                continue

            face_resized = cv2.resize(face_img, (200, 200))
            valid_images.append(face_resized)

        print(f"{person}: {len(valid_images)} original images retained")

        augmented_images = valid_images.copy()
        i = 0
        while len(augmented_images) < target_images_per_person:
            augmented = augment_image(valid_images[i % len(valid_images)])
            for img in augmented:
                if len(augmented_images) >= target_images_per_person:
                    break
                augmented_images.append(img)
            i += 1

        person_images[current_label] = augmented_images[:target_images_per_person]
        print(f"{person}: augmented to {len(person_images[current_label])} images")
        current_label += 1

    images = []
    labels = []

    for label, img_list in person_images.items():
        images.extend(img_list)
        labels.extend([label] * len(img_list))

    return images, np.array(labels), label_dict

class DCVAHybridClassifier:
    def __init__(self, eigenfaces, mean_vector):
        self.eigenfaces = eigenfaces
        self.mean = mean_vector
        self.class_vectors = {}
        self.labels = []
        self.label_names = {}

    def fit(self, faces, labels, label_names):
        self.label_names = label_names
        features = []
        for img in faces:
            flat = img.flatten().astype(np.float32).reshape(1, -1)
            proj = (flat - self.mean) @ self.eigenfaces
            features.append(proj.flatten())

        features = np.array(features)
        labels = np.array(labels)

        class_features = defaultdict(list)
        for feat, lbl in zip(features, labels):
            class_features[lbl].append(feat)

        for lbl, feats in class_features.items():
            feats = np.array(feats)
            mean_vector = np.mean(feats, axis=0)
            residuals = feats - mean_vector
            U, S, Vt = np.linalg.svd(residuals, full_matrices=False)
            null_space = Vt[S < 1e-5]
            if null_space.size == 0:
                common_vec = mean_vector
            else:
                proj = residuals @ null_space.T @ null_space
                common_vec = mean_vector + proj.mean(axis=0)
            self.class_vectors[lbl] = normalize(common_vec.reshape(1, -1))[0]

    def predict(self, img, method="cosine"):
        flat = img.flatten().astype(np.float32).reshape(1, -1)
        proj = (flat - self.mean) @ self.eigenfaces
        test_vec = normalize(proj)[0]

        best_label = None
        best_score = float("inf") if method == "euclidean" else -1

        for lbl, class_vec in self.class_vectors.items():
            if method == "cosine":
                score = np.dot(test_vec, class_vec)
                if score > best_score:
                    best_score = score
                    best_label = lbl
            else:
                score = euclidean(test_vec, class_vec)
                if score < best_score:
                    best_score = score
                    best_label = lbl

        if best_label is None:
            return None, 0.0
        else:
            return best_label, np.clip(best_score, 0.0, 1.0)

faces, labels, label_dict = load_training_data("png_dataset/png_dataset")
recognizer = cv2.face.FisherFaceRecognizer_create()
recognizer.train(faces, labels)
eigenfaces = recognizer.getEigenVectors()
mean = recognizer.getMean().reshape(1, -1)

dcva = DCVAHybridClassifier(eigenfaces, mean)
dcva.fit(faces, labels, label_dict)

cap = cv2.VideoCapture(0)
if not cap.isOpened():
    print("Error: Cannot open webcam.")
    exit()

print("Press 'q' to exit.")

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame.")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    processed = preprocess_face(gray)
    detected_faces = hog_detector(processed)

    for face in detected_faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        x, y = max(0, x), max(0, y)
        x2, y2 = min(gray.shape[1], x + w), min(gray.shape[0], y + h)

        face_img = processed[y:y2, x:x2]
        if face_img.size == 0:
            continue

        face_resized = cv2.resize(face_img, (200, 200))
        label, score = dcva.predict(face_resized, method="cosine")
        threshold = 0.5

        if label is not None and score > threshold:
            name = label_dict.get(label, "Unknown")
        else:
            name = "Unknown"

        cv2.rectangle(frame, (x, y), (x2, y2), (0, 255, 0), 2)
        cv2.putText(frame, f"{name} ({score:.2f})", (x, y - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    cv2.imshow("Fisherface + DCVA Recognition (Press 'q' to quit)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

amisha: 30 original images retained
amisha: augmented to 35 images
dhanoosh: 28 original images retained
dhanoosh: augmented to 35 images
jose: 35 original images retained
jose: augmented to 35 images
jui: 29 original images retained
jui: augmented to 35 images
ritvi: 36 original images retained
ritvi: augmented to 35 images
siddhangana: 26 original images retained
siddhangana: augmented to 35 images
sparsh: 36 original images retained
sparsh: augmented to 35 images
yash: 22 original images retained
yash: augmented to 35 images
Press 'q' to exit.
